<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2c6405fa6fe54ec860e54f9dfeea5ba54a5b7bbb4694ec67b79be3257b4122b8
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 11:11:57
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 13.20 K (0.09%)
Current PnL: -20.98 L (-13.92%)
CY Booked + Current PnL: -9.54 L (-6.33%)
-------------------
Total profit:  1.62 L
Total loss:  -22.60 L
-------------------
Total Booked + Current PnL: 17.44 L (14.1%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.49 L (62.93%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122086.0,21582.0,7520.0,3.01,21.47,6.16,28.96,79.0,2.87,0.87,62.73,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,40.0,M-SC,1.00,83447.0,-17330.0,17424.0,0.27,-17.20,20.88,0.09,245.0,-0.99,0.60,9.63,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.52,45.0,H-LC,3.67,207185.0,2685.0,20615.0,1.18,1.31,9.95,11.39,10.0,0.13,1.48,10.38,X40N,ATH,PHARMA
14,BLUESTARCO,2080.00,8.25,44.0,H-MC,2.99,186990.0,22320.0,21018.0,-1.19,13.55,11.24,26.31,89.0,1.06,1.34,22.62,X40N,NTT,AC
84,VOLTAS,1530.00,-1.08,53.0,H-MC,1.18,207630.0,15888.0,21863.0,1.24,8.29,10.53,19.69,99.0,0.73,1.48,15.65,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,36.19,74.0,M-LC,7.36,221698.0,16622.0,95951.0,3.90,8.11,43.28,54.89,52.0,0.17,1.58,34.15,X5K,ATH,METALS
60,SAIL,228.00,49.16,61.0,M-MC,12.66,234846.0,9884.0,157770.0,3.54,4.39,67.18,74.53,192.0,0.06,1.68,38.06,XY24,BTT,STEEL
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122086.0,21582.0,7520.0,3.01,21.47,6.16,28.96,79.0,2.87,0.87,62.73,MH,ATH,METALS
67,SONACOMS,1006.00,-41.58,44.0,M-SC,13.81,72712.0,-28447.0,103338.0,2.53,-28.12,142.12,74.03,202.0,-0.28,0.52,2.53,AR,BTT,AUTO
55,RAJOOENG,143.10,-41.44,42.0,H-SC,16.47,88770.0,-13730.0,54327.0,2.15,-13.40,61.20,39.61,114.0,-0.25,0.63,8.41,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TITAGARH,1548.00,-4.07,51.0,H-SC,4.72,204972.0,-37205.0,147969.0,-3.05,-15.36,72.19,45.74,158.0,-0.25,1.46,33.31,XY24,NTT,ENGINEERING
12,BATAINDIA,2096.00,-34.87,44.0,M-SC,4.40,97442.0,-31228.0,76521.0,-2.25,-24.27,78.53,35.20,219.0,-0.41,0.70,11.84,X40,NTT,FOOTWEAR
53,QUESS,986.00,-48.21,24.0,M-SC,37.86,51465.0,-13541.0,164467.0,-1.96,-20.83,319.57,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
58,REPCOHOME,880.00,-59.02,57.0,H-SC,2.76,235334.0,-51171.0,308499.0,-1.94,-17.86,131.09,89.82,134.0,-0.17,1.68,24.51,XY24,NTT,FINANCE
52,PGHH,17907.65,-27.37,57.0,H-MC,2.04,210315.0,9495.0,58299.0,-1.87,4.73,27.72,33.76,80.0,0.16,1.50,10.17,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-13.34,49.0,M-LC,5.67,152136.0,1191.0,108260.0,0.45,0.79,71.16,72.51,53.0,0.01,1.09,6.86,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,62.0,H-SC,13.55,130707.0,-11772.0,31291.0,1.49,-8.26,23.94,13.70,163.0,-0.38,0.93,54.27,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,40.0,M-SC,1.00,83447.0,-17330.0,17424.0,0.27,-17.20,20.88,0.09,245.0,-0.99,0.60,9.63,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,1.97,220617.0,-49050.0,85379.0,0.22,-18.19,38.70,13.47,138.0,-0.57,1.58,12.18,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.37,53.0,H-MC,6.52,105127.0,-26708.0,68932.0,1.34,-20.26,65.57,32.03,98.0,-0.39,0.75,17.75,OX40N,NTT,IT
18,CERA,9475.0,-23.39,38.0,H-SC,1.67,139633.0,-36270.0,78292.0,-1.72,-20.62,56.07,23.89,149.0,-0.46,1.00,20.59,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122086.0,21582.0,7520.0,3.01,21.47,6.16,28.96,79.0,2.87,0.87,62.73,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,61.0,M-MC,12.66,234846.0,9884.0,157770.0,3.54,4.39,67.18,74.53,192.0,0.06,1.68,38.06,XY24,BTT,STEEL
32,HINDZINC,730.22,36.19,74.0,M-LC,7.36,221698.0,16622.0,95951.0,3.90,8.11,43.28,54.89,52.0,0.17,1.58,34.15,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.25,44.0,H-MC,2.99,186990.0,22320.0,21018.0,-1.19,13.55,11.24,26.31,89.0,1.06,1.34,22.62,X40N,NTT,AC
84,VOLTAS,1530.00,-1.08,53.0,H-MC,1.18,207630.0,15888.0,21863.0,1.24,8.29,10.53,19.69,99.0,0.73,1.48,15.65,XY25,NTT,AC
17,CAMS,5211.76,-6.49,44.0,H-SC,1.66,106238.0,4234.0,39691.0,0.37,4.15,37.36,43.06,122.0,0.11,0.76,21.98,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122086.0,21582.0,7520.0,3.01,21.47,6.16,28.96,79.0,2.87,0.87,62.73,MH,ATH,METALS
32,HINDZINC,730.22,36.19,74.0,M-LC,7.36,221698.0,16622.0,95951.0,3.90,8.11,43.28,54.89,52.0,0.17,1.58,34.15,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.51,193665.0,13857.0,98556.0,0.87,7.71,50.89,62.52,88.0,0.14,1.38,44.42,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.08,45.0,H-SC,8.95,128059.0,4723.0,126906.0,-0.88,3.83,99.10,106.72,119.0,0.04,0.91,28.10,AR,ATH,MISC
86,WIPRO,420.00,-13.34,49.0,M-LC,5.67,152136.0,1191.0,108260.0,0.45,0.79,71.16,72.51,53.0,0.01,1.09,6.86,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.41,276350.0,-39492.0,152158.0,-0.39,-12.50,55.06,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205330.0,-46438.0,99647.0,0.21,-18.44,48.53,21.13,27.0,-0.47,1.47,10.45,X40,ATH,PAINTS
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245280.0,-16339.0,135542.0,-0.36,-6.25,55.26,45.57,7.0,-0.12,1.75,4.37,AR,NTT,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.34,42.0,H-LC,8.02,251560.0,-5503.0,40652.0,0.62,-2.14,16.16,13.67,24.0,-0.14,1.80,17.83,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-0.90,50.0,H-LC,1.95,159250.0,-26845.0,74322.0,-0.99,-14.43,46.67,25.51,15.0,-0.36,1.14,17.35,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
48,LTIM,7230.2,-2.60,59.0,H-LC,2.52,241245.0,-4906.0,84122.0,0.35,-1.99,34.87,32.18,16.0,-0.06,1.72,33.33,X200,ATH,IT
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,2.61,276237.0,10811.0,45993.0,0.00,4.07,16.65,21.40,11.0,0.24,1.97,10.99,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,49.0,H-LC,3.10,215015.0,4649.0,24125.0,0.80,2.21,11.22,13.68,37.0,0.19,1.54,19.10,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.41,276350.0,-39492.0,152158.0,-0.39,-12.50,55.06,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245280.0,-16339.0,135542.0,-0.36,-6.25,55.26,45.57,7.0,-0.12,1.75,4.37,AR,NTT,ELECTRICAL
80,UNITDSPR,1644.00,-12.37,51.0,H-LC,6.67,229706.0,-4438.0,53062.0,0.18,-1.90,23.10,20.77,86.0,-0.08,1.64,4.89,X40N,NTT,BREWERIES
74,TCS,4406.34,-28.36,52.0,H-LC,12.18,285149.0,-60807.0,129058.0,0.21,-17.58,45.26,19.73,1.0,-0.47,2.04,5.02,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.90,50.0,H-LC,1.95,159250.0,-26845.0,74322.0,-0.99,-14.43,46.67,25.51,15.0,-0.36,1.14,17.35,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205330.0,-46438.0,99647.0,0.21,-18.44,48.53,21.13,27.0,-0.47,1.47,10.45,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.52,45.0,H-LC,3.67,207185.0,2685.0,20615.0,1.18,1.31,9.95,11.39,10.0,0.13,1.48,10.38,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.04,49.0,H-LC,3.10,215015.0,4649.0,24125.0,0.80,2.21,11.22,13.68,37.0,0.19,1.54,19.10,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,52.0,H-LC,12.18,285149.0,-60807.0,129058.0,0.21,-17.58,45.26,19.73,1.0,-0.47,2.04,5.02,X40,ATH,IT
40,INFY,2275.00,-17.80,55.0,H-LC,7.91,319564.0,6676.0,165023.0,0.35,2.13,51.64,54.87,3.0,0.04,2.28,8.92,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
83,VBL,671.64,-23.20,28.0,H-LC,10.41,276350.0,-39492.0,152158.0,-0.39,-12.50,55.06,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245280.0,-16339.0,135542.0,-0.36,-6.25,55.26,45.57,7.0,-0.12,1.75,4.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,29.26,79104.0,-34445.0,74492.0,1.30,-30.33,94.17,35.27,268.0,-0.46,0.56,93.61,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,40.0,H-SC,10.60,90285.0,-7695.0,29297.0,-1.00,-7.85,32.45,22.05,152.0,-0.26,0.64,31.24,XR,ATH,FINANCE
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122086.0,21582.0,7520.0,3.01,21.47,6.16,28.96,79.0,2.87,0.87,62.73,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,62.0,H-SC,13.55,130707.0,-11772.0,31291.0,1.49,-8.26,23.94,13.70,163.0,-0.38,0.93,54.27,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,14.38,144945.0,-4275.0,148453.0,1.24,-2.86,102.42,96.62,208.0,-0.03,1.04,56.86,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,14.38,144945.0,-4275.0,148453.0,1.24,-2.86,102.42,96.62,208.0,-0.03,1.04,56.86,XY25,NTT,FINANCE
32,HINDZINC,730.22,36.19,74.0,M-LC,7.36,221698.0,16622.0,95951.0,3.90,8.11,43.28,54.89,52.0,0.17,1.58,34.15,X5K,ATH,METALS
69,SURYODAY,240.00,60.09,60.0,H-SC,7.97,147655.0,-31416.0,97142.0,0.11,-17.54,65.79,36.71,135.0,-0.32,1.05,46.50,XR,NTT,BANKS
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122086.0,21582.0,7520.0,3.01,21.47,6.16,28.96,79.0,2.87,0.87,62.73,MH,ATH,METALS
26,GREENPANEL,537.00,231.40,58.0,M-SC,4.88,147937.0,-35141.0,114119.0,-1.33,-19.19,77.14,43.14,230.0,-0.31,1.06,39.16,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.47
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.95
LC    32.31
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.97
X40      14.66
X40N     11.16
XR       10.90
XY25     10.43
AR        8.15
OX40N     7.71
X200      1.72
MH        1.63
X5K       1.58
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.91
H-LC    25.75
H-MC    20.43
M-SC    13.63
M-LC     5.54
M-MC     3.02
L-SC     1.41
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.68,-7.09,42.39
IT,12.88,-16.61,79.76
FINANCE,10.49,-14.92,70.42
MISC,6.91,-18.81,84.64
BANKS,6.33,-12.49,71.66
PAINTS,5.51,-19.81,37.66
ELECTRICAL,5.33,-10.29,48.99
INSURANCE,3.83,-3.35,41.99
AC,3.51,1.49,15.01


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3237785.0,22
XR,1382115.0,14
AR,1188620.0,9
X40,789246.0,10
X40N,673372.0,9
OX40N,568231.0,10
XY25,478518.0,7
SR,265228.0,2
X5K,95951.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3741946.0,29
M-SC,1525448.0,17
H-MC,1291322.0,15
H-LC,1190147.0,15
M-LC,374920.0,4
M-MC,361229.0,2
L-SC,262430.0,3
L-MC,59869.0,1
L-LC,41487.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206361.0      6
M-SC       XY24       852090.0      7
H-SC       AR         809881.0      5
           XR         808905.0      7
H-MC       XY24       578634.0      4
H-LC       X40        490794.0      5
M-MC       XY24       361229.0      2
H-SC       X40N       309596.0      4
           SR         265228.0      2
           OX40N      263885.0      4
H-LC       X40N       225835.0      3
H-MC       X40        221931.0      4
H-LC       AR         209864.0      2
H-MC       XY25       177808.0      2
L-SC       XR         173267.0      2
M-LC       XY24       170709.0      2
M-SC       AR         168875.0      2
           XY25       148453.0      1
           OX40N      146251.0      4
H-MC       X40N       137941.0      2
M-SC       XR         133258.0      2
H-LC       XY25       110770.0      3
M-LC       XR         108260.0      1
H-MC       XR          98556.0      1
M-LC       X5K         95951.0      1
L-SC       OX40N       89163.0      1
H-LC       X200        84122.0      1
H-SC       MH          78090.0      1
M-SC       X40         76521.0      1
H-MC       OX40N       68932.0      1
H-LC       XY24        68762.0      1
L-MC       XR          59869.0      1
L-LC       XY25        41487.0      1
H-MC       MH           7520.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
